# 13장 - 모델 성능 검증하기

### 1. 데이터 확인과 예측 실행

In [ ]:
# 케라스 api 호출
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 라이브러리 호출
import pandas as pd

In [ ]:
!git clone https://github.com/taehojo/data.git

df = pd.read_csv('./data/sonar3.csv', header=None)

df.head() # 상위 5개 행 확인
# 전체 61개 열, 마지막 열은 광물의 종류를 나타냄 (일반 암석 : 0, 광석 : 1)

fatal: destination path 'data' already exists and is not an empty directory.


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [ ]:
# 일반 암석과 광석의 갯수 세기
df[60].value_counts()

# 광석 111개, 암석 97개

1    111
0     97
Name: 60, dtype: int64

In [ ]:
X = df.iloc[:,0:60] # 1~60번째 열
y = df.iloc[:, 60] # 광물의 종류

### 2. 딥러닝 실행

In [ ]:
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 실행
history = model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
21/21 [==============================] - 1s 2ms/step - loss: 0.7574 - accuracy: 0.4952
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6820 - accuracy: 0.5769
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6631 - accuracy: 0.6298
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6362 - accuracy: 0.6587
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6212 - accuracy: 0.6635
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6010 - accuracy: 0.7212
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5831 - accuracy: 0.7163
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5651 - accuracy: 0.7356
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5452 - accuracy: 0.7548
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5297 - accuracy: 0.7788
Epoch 11/

### 3. 과적합 이해하기
- 모델이 학습 데이터셋 안에서는 일정 수준 이상의 예측 정확도를 보이지만, 새로운 데이터에 적용하면 잘 맞지 않는 것을 의미함
  - 즉, 주어진 샘플에만 최적화되어있어, 새로운 데이터에 이 기준을 적용하면 정확한 분류가 어려워진다는 뜻
- 층이 너무 많거나 변수가 복잡해서 발생하기도 하고, 테스트셋과 학습셋이 중복될 때 생기기도 함
  - 딥러닝에서는 입력층, 은닉층, 출력층의 노드들에 상당히 많은 변수가 투입되므로, 과적합을 주의해야 함



### 4. 과적합 방지하기
- 학습을 하는 데이터셋과 이를 테스트할 데이터셋을 완전히 구분
- 학습과 동시에 테스트를 병행하며 진행
- e.g. 총 100개의 샘플 데이터셋 -> 70개는 학습셋 / 30개는 테스트셋으로 분류
  - 70개의 샘플로 학습을 진행한 결과를 모델로 저장 후, 나머지 30개의 샘플로 테스트해 정확도를 살펴보면 됨

- 머신 러닝의 최종 목적은 `과거의 데이터를 토대로 새로운 데이터를 예측하는 것`
  - 즉, 새로운 데이터에 사용할 모델을 만드는 것이 최종 목적이므로 테스트셋을 만들어 정확한 평가를 병행하는 것이 매우 중요
  - 학습이 깊어져 학습셋 내부에서 성공률이 높아져도, 테스트셋에서 효과가 없어 테스트 에러가 커지는 순간, 과적합 발생
- `train_test_split()`: 사이킷런 라이브러리에서 제공하는 정해진 비율로 학습셋과 테스트셋 분리시키는 함수

In [ ]:
# 학습셋과 테스트셋 구분
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 실행
history = model.fit(X_train, y_train, epochs=200, batch_size=10)

Epoch 1/200
15/15 [==============================] - 1s 2ms/step - loss: 0.6828 - accuracy: 0.5793
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6632 - accuracy: 0.5931
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6467 - accuracy: 0.6207
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6327 - accuracy: 0.6069
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6169 - accuracy: 0.6621
Epoch 6/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6089 - accuracy: 0.7034
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 0.5918 - accuracy: 0.6828
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 0.5824 - accuracy: 0.6897
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 0.5681 - accuracy: 0.7448
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 0.5638 - accuracy: 0.6966
Epoch 11/

In [ ]:
# 앞서 만든 구조의 모델 유지 후, 테스트 함수를 추가
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

2/2 [==============================] - 0s 7ms/step - loss: 0.5322 - accuracy: 0.8730
Test accuracy:  0.8730158805847168


### 5. 모델 저장과 재사용
- model.save() 함수로 모델 이름 저장
- hdf5 포맷 -> 크고 복잡한 데이터 저장
- 케라스 api의 load_model 함수를 사용해 이를 다시 불러올 수 있음



In [ ]:
# 모델 저장 (이름, 위치)
model.save('./data/model/my_model.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 모델 불러오기
from tensorflow.keras.models import Sequential, load_model

In [ ]:
# 모델 삭제
del model

In [ ]:
# 모델 불러오기
model = load_model('./data/model/my_model.hdf5')

# 불러온 모델에 테스트셋 적용해 정확도 구하기 -> 이전과 동일
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

2/2 [==============================] - 0s 10ms/step - loss: 0.5322 - accuracy: 0.8730
Test accuracy:  0.8730158805847168


### 6. k겹 교차 검증
- 실제 프로젝트에서 학습셋을 70%, 테스트셋을 30%로 설정할 경우, 30%는 학습에 이용할 수 없다는 단점 존재
- Solved by `k-fold cross validation`
  - 데이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고, 나머지를 모두 합해 학습셋으로 사용하는 방법
  - 가지고 있는 데이터의 100%를 학습셋으로 사용가능하며, 동시에 테스트셋으로도 사용할 수 있음
- 5겹 교차 검증
  - 데이터셋을 다섯 개로 나눈 후, 그 중 4개를 학습셋으로, 나머지 하나를 테스트셋으로 만들어 다섯 번의 학습을 순차적으로 실시
  - 사이킷런 라이브러리의 `KFold()`함수 사용



In [ ]:
# 5겹 교차 검증
from sklearn.model_selection import KFold
k = 5 # 5개의 파일로 나누기
kfold = KFold(n_splits=k, shuffle=True)
acc_score = [] # 정확도가 채워질 빈 리스트

In [ ]:
def model_fn():
  model = Sequential()
  model.add(Dense(24, input_dim=60, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
for train_index, test_index in kfold.split(X): # 학습셋, 테스트셋으로 분류되며, k번 반복
  X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
model = model_fn()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=0)

In [ ]:
accuracy = model.evaluate(X_test, y_test)[1] # 정확도 구하기
acc_score.append(accuracy) # 리스트에 저장

2/2 [==============================] - 0s 9ms/step - loss: 0.6230 - accuracy: 0.8537


In [ ]:
# k번 실시된 정확도의 평균
avg_acc_score = sum(acc_score) / k

# 결과 출력
print('정확도: ', acc_score)
print('정확도 평균: ', avg_acc_score)

정확도:  [0.8536585569381714]
정확도 평균:  0.17073171138763427
